In [1]:
import sys

sys.path.append('./helpers_models/')
sys.path.append('./data_visualization_and_augmentations/')
sys.path.append('../torch_videovision/')
sys.path.append('./important_csvs/')
sys.path.append('../video-classification/ResNetCRNN/')

from helpers_lstm import *
from helpers_training import *


In [2]:
tensor_transform = get_tensor_transform('ImageNet', True)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
df = pd.read_csv('./small_dataset_csvs/events_with_number_of_frames_stratified.csv')
df_train = get_df(df, 20, True, False, False)
class_image_paths, end_idx = get_indices(df_train, root_dir)
train_loader = get_loader(16, 128, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, True, False)
df_valid = get_df(df, 20, False, True, False)
class_image_paths, end_idx = get_indices(df_valid, root_dir)
valid_loader = get_loader(16, 128, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, True, False)
df_test = get_df(df, 20, False, False, True)
class_image_paths, end_idx = get_indices(df_test, root_dir)
test_loader = get_loader(16, 128, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, True, False)


In [3]:
device = torch.device('cuda')
cnn_encoder = ResCNNEncoder().to(device)
adaptive_pool = AdaptiveConcatPool2d()
cnn_encoder.resnet[8] = adaptive_pool
for param in cnn_encoder.parameters():
    param.requires_grad = False
for param in cnn_encoder.resnet[8].parameters():
    param.requires_grad = True
for param in cnn_encoder.headbn1.parameters():
    param.requires_grad = True
for param in cnn_encoder.fc1.parameters():
    param.requires_grad = True
    
rnn_decoder = DecoderRNNattention(batch_size=128).to(device)
for param in rnn_decoder.parameters():
    param.requires_grad = True

In [7]:
crnn_params, cnn_encoder, rnn_decoder = parallelize_model(cnn_encoder, rnn_decoder)

model = nn.Sequential(cnn_encoder,rnn_decoder)
torch.cuda.empty_cache()

Using 4 GPUs!


In [8]:
checkpoint = torch.load('/media/scratch/astamoulakatos/saved-lstm-models/best-checkpoint-004epoch.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print('loading pretrained freezed model!')


loading pretrained freezed model!


In [9]:
unfreeze(model[0].module ,0.6)

unfreezing these layer ['headbn1', 'headdr1', 'fc1']


In [10]:
unfreeze(model[0].module.resnet ,0.3)

unfreezing these layer ['6', '7', '8']


In [11]:
unfreeze(model[1].module, 1)

unfreezing these layer ['LSTM', 'fc1']


In [12]:
check_freeze(model[0].module)

resnet False
headbn1 True
headdr1 True
fc1 True


In [13]:
check_freeze(model[0].module.resnet)

0 False
1 False
2 True
3 True
4 False
5 False
6 True
7 True
8 True


In [14]:
check_freeze(model[1].module)

LSTM True
fc1 True
